## 필요사항 설치

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 16.3 MB/s 
     |████████████████████████████████| 163 kB 75.6 MB/s 
     |████████████████████████████████| 7.6 MB 66.4 MB/s 


In [2]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 441 kB 27.0 MB/s 
     |████████████████████████████████| 95 kB 3.9 MB/s 
     |████████████████████████████████| 212 kB 90.2 MB/s 
     |████████████████████████████████| 115 kB 77.1 MB/s 
     |████████████████████████████████| 127 kB 69.7 MB/s 
     |████████████████████████████████| 115 kB 71.2 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: dill
    Found existing installation: dill 0.3.6
    Uninstalling dill-0.3.6:
      Successfully uninstalled dill-0.3.6


In [3]:
!pip install wandb -qU

     |████████████████████████████████| 1.9 MB 34.8 MB/s 
     |████████████████████████████████| 182 kB 16.6 MB/s 
     |████████████████████████████████| 166 kB 87.0 MB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 
     |████████████████████████████████| 166 kB 56.6 MB/s 
     |████████████████████████████████| 162 kB 75.3 MB/s 
     |████████████████████████████████| 162 kB 77.5 MB/s 
     |████████████████████████████████| 158 kB 94.2 MB/s 
     |████████████████████████████████| 157 kB 95.1 MB/s 
     |████████████████████████████████| 157 kB 64.1 MB/s 
     |████████████████████████████████| 157 kB 92.0 MB/s 
     |████████████████████████████████| 157 kB 78.7 MB/s 
     |████████████████████████████████| 157 kB 58.6 MB/s 
     |████████████████████████████████| 157 kB 89.5 MB/s 
     |████████████████████████████████| 157 kB 48.9 MB/s 
     |████████████████████████████████| 156 kB 90.5 MB/s 


In [40]:
!git clone https://github.com/cmj-dev/groomProject.git

Cloning into 'groomProject'...
remote: Enumerating objects: 252, done.
remote: Counting objects: 100% (87/87), done.
remote: Compressing objects: 100% (68/68), done.
remote: Total 252 (delta 39), reused 50 (delta 18), pack-reused 165
Receiving objects: 100% (252/252), 6.67 MiB | 8.32 MiB/s, done.
Resolving deltas: 100% (108/108), done.


## 데이터 전처리

In [7]:
import wandb
wandb.login()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

## readjson

In [9]:
from datasets import load_dataset
datasets = load_dataset('smilegate-ai/kor_unsmile')

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/smilegate-ai___parquet/smilegate-ai--kor_unsmile-1dba960877497f9f/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [10]:
datasets

DatasetDict({
    train: Dataset({
        features: ['문장', '여성/가족', '남성', '성소수자', '인종/국적', '연령', '지역', '종교', '기타 혐오', '악플/욕설', 'clean', '개인지칭', 'labels'],
        num_rows: 15005
    })
    valid: Dataset({
        features: ['문장', '여성/가족', '남성', '성소수자', '인종/국적', '연령', '지역', '종교', '기타 혐오', '악플/욕설', 'clean', '개인지칭', 'labels'],
        num_rows: 3737
    })
})

In [14]:
test_data = datasets['train']

In [30]:
test_data = test_data.remove_columns(column_names=['여성/가족', '남성', '성소수자', '인종/국적', '연령', '지역', '종교', '기타 혐오', '악플/욕설', '개인지칭', 'labels'])

In [31]:
test_data.to_csv('evaluationData.csv')

Creating CSV from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

1623137

In [36]:
def makeLabel(data):
    result = {}
    result['texts'] = data['문장']
    result['labels'] = data['clean']*(-1) + 1
    return result

In [37]:
labeled_data = test_data.map(makeLabel, remove_columns=test_data.column_names)

  0%|          | 0/15005 [00:00<?, ?ex/s]

In [38]:
labeled_data

Dataset({
    features: ['texts', 'labels'],
    num_rows: 15005
})

## make generated data

In [ ]:
from groomProject.src import generaterMethod, modelLoader

def get_model():
    art1='groom2team/pj3_classifier_data_add/pytorch_finetuned:v2'
    art1model='beomi/KcELECTRA-base-v2022'
    art2model='skt/kogpt2-base-v2'
    art2='groom2team/pj3_generater_gpt2/pytorch_finetuned:v1'

    lmodel,ltokenizer=modelLoader.loadModelForSequenceClassification(art1model,art1)
    gmodel,gtokenizer=modelLoader.loadModelForCausalLM(art2model,art2)

    return (lmodel,ltokenizer,gmodel,gtokenizer)

lmodel,ltokenizer,gmodel,gtokenizer = get_model()
emodel=generaterMethod.generater(lmodel,ltokenizer,gmodel,gtokenizer)



In [42]:
def generateData(data):
    result = {}
    result['generated'] = emodel.makeMoralText(data['texts'])
    result['original'] = data['texts']
    return result

In [ ]:
evaluation_data = labeled_data.map(generateData, remove_columns=labeled_data.column_names)

In [ ]:
evaluation_data.to_csv('eval.csv')